In [9]:
import numpy as np
import IPython.display as ipd
import librosa
import librosa.display
import matplotlib.pyplot as plt
import utils
import os
import pandas as pd
import _pickle as pickle
%matplotlib inline
AUDIO_DIR = '/media/ravi/507412DD7412C59E/fma_small/'


In [2]:

# Load metadata and features.
tracks = utils.load('/home/ravi/metafma/fma_metadata/tracks.csv')
genres = utils.load('/home/ravi/metafma/fma_metadata/genres.csv')
features = utils.load('/home/ravi/metafma/fma_metadata/features.csv')
echonest = utils.load('/home/ravi/metafma/fma_metadata/echonest.csv')

/home/ravi/Amadeus/fma_code/utils.py:214: FutureWarning: specifying 'categories' or 'ordered' in .astype() is deprecated; pass a CategoricalDtype instead
  'category', categories=SUBSETS, ordered=True)


In [72]:
tracks.loc[3792]

album   comments                                                             0
        date_created                                       2008-12-04 09:27:18
        date_released                                      2008-04-03 00:00:00
        engineer                                                    Jesse Chan
        favorites                                                            0
        id                                                                 883
        information          <p>Live at WFMU on Pat Duncan's show on April ...
        listens                                                           1558
        producer                                                    Pat Duncan
        tags                                                                []
        title                    Live at WFMU on Pat Duncan's Show on 4/3/2008
        tracks                                                               9
        type                                        

In [3]:
filename = utils.get_audio_path(AUDIO_DIR,2)
print('File: {}'.format(filename))

x = utils.FfmpegLoader(sampling_rate=44100)._load(filename,"/home/ravi/anaconda3/envs/amadeus/bin/")
sr=44100
c=4
xr=librosa.core.resample(x,sr,int(sr/c))
sr = int(sr/c)

File: /media/ravi/507412DD7412C59E/fma_small/000/000002.mp3


In [54]:
xr.shape

(330492,)

In [55]:
xr[start*sr:end*sr]

array([ 6856,  7221,  5510, ..., -1211, -8590,     0], dtype=int16)

In [56]:
start, end = 1, 30
ipd.Audio(data=xr[start*sr:end*sr], rate=sr)

In [75]:
def id_from_genre_text(genre_text,genres,top_level=True):
    if top_level:
        return genres.loc[genres['title']==genre_text].iloc[0]["top_level"]
    return genres.index[genres['title']==genre_text].tolist()[0]

def get_genre_id(track_id,tracks,genres):
    try:
        track_genre_title = tracks[('track','genre_top')]
        if track_genre_title is str:
            return id_from_genre_text(track_genre_title,genres)
        else:
            return tracks[('track','genres')].loc[track_id][0]
    except:
        print( tracks["set","subset"].iloc[track_id])
        raise

In [77]:
get_genre_id(3792,tracks,genres)

12

In [39]:
small = tracks['set', 'subset'] <= 'small'


In [40]:
small.shape

(106574,)

In [41]:
small = tracks['set', 'subset'] <= 'small'

train = tracks['set', 'split'] == 'training'
val = tracks['set', 'split'] == 'validation'
test = tracks['set', 'split'] == 'test'

y_train = tracks.loc[small & train, ('track', 'genre_top')].apply(id_from_genre_text,args=(genres,))
y_test = tracks.loc[small & test, ('track', 'genre_top')].apply(id_from_genre_text,args=(genres,))

X_train = 
X_test = features.loc[small & test, 'mfcc']

print('{} training examples, {} testing examples'.format(y_train.size, y_test.size))
print('{} features, {} classes'.format(X_train.shape[1], np.unique(y_train).size))

SyntaxError: invalid syntax (<ipython-input-41-55fd7b379785>, line 10)

In [74]:
train

NameError: name 'train' is not defined

In [112]:
out_ad = '/home/ravi/test_samples/'

In [113]:
if not os.path.exists(out_ad):
    os.makedirs(out_ad)

In [114]:
def make_csvs(track_id_prefix,out_dir,genres_df,tracks_df):
    out_file_path = out_dir+"/{}.p".format(track_id_prefix)
    raw_songs = dict()
    genres = dict()
    song_directory = os.fsencode(AUDIO_DIR+"{:03d}".format(track_id_prefix))
    out_str = "track_id,genre_id,raw_song\n"
    for file in os.listdir(song_directory):
    
            filename = AUDIO_DIR+"{:03d}/".format(track_id_prefix)+ os.fsdecode(file)
            if filename.endswith(".mp3"):
                track_id = int(os.fsdecode(file)[:-4])
                #print(track_id)
    #             print(tracks_df.loc[track_id])
    #             if tracks_df["set","subset"].iloc[track_id]!="small":
    #                 #print(tracks_df.iloc[track_id])
    #                 continue
                try:
                    x = utils.FfmpegLoader(sampling_rate=44100)._load(filename,"/home/ravi/anaconda3/envs/amadeus/bin/")
                except:
                    print("bad song {}".format(track_id))
                    continue
                sr=44100
                c=4
                xr=librosa.core.resample(x,sr,int(sr/c))


                raw_songs[track_id] = pd.Series(xr)

                genres[track_id] = get_genre_id(track_id,tracks_df,genres_df)
            #song_str = ""
#             for i in raw_songs[track_id]:
#                 song_str+="{},".format(i)
#             out_str += "{};{};{}\n".format(track_id,genres[track_id],song_str)
    df = pd.DataFrame().from_dict({"genres":genres,"raw_songs":raw_songs})
#     with open(out_file_path,'w') as outfile:
#         outfile.write(out_str)
    
    #df.to_csv(out_file_path)
#     with open(out_file_path,"wb") as outfile:
#         pickle.dump(df,outfile)
    df.to_pickle(out_file_path)
    
    return df

In [ ]:
out_str = "track_id,genre_id,raw_song\n"
for #loop through df here:
    raw_songs[track_id] = pd.Series(xr)
    genres[track_id] = get_genre_id(track_id,tracks_df,genres_df)
    song_str = ""
    for i in raw_songs[track_id]:
        song_str+="{},".format(i)
    out_str += "{};{},{}\n".format(track_id,genres[track_id],song_str)

In [116]:
for i in [file[1] for file in os.walk(AUDIO_DIR)][0]:
    
    if not os.path.exists(out_ad+"/{}.p".format(int(i))):
        print(int(i))
        make_csvs(int(i),out_ad,genres,tracks)
    

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43


KeyboardInterrupt: 

In [59]:
a.to_pickle()

,genres,raw_songs
2012,17,0 0 1 0 2 ...
2096,12,0 0 1 0 2 ...
2097,12,0 0 1 0 2 ...
2099,12,0 0 1 0 2 ...


In [28]:
a.shape

(98, 2)

In [73]:
a.iloc[0]

genres                                                      17
raw_songs    [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...
Name: 2012, dtype: object

In [ ]:
a.to_csv()

In [80]:
b=pd.read_csv(out_ad+"2test.csv")

In [82]:
pd.Series(b['raw_songs'].iloc[0])

0    0             0\n1             0\n2           ...
dtype: object

In [86]:
import time
st = time.time()
with open("/home/ravi/Amadeus/test_samples/75test.p",'rb') as infile:
    d = pickle.load(infile)
print( time.time()-st)
    

0.6485109329223633


In [ ]:
def load_track(filename, enforce_shape=None):
    new_input, sample_rate = lbr.load(filename, mono=True)
    features = librosa.feature.melspectrogram(new_input, **MEL_KWARGS).T

    if enforce_shape is not None:
        if features.shape[0] < enforce_shape[0]:
            delta_shape = (enforce_shape[0] - features.shape[0],
                    enforce_shape[1])
            features = np.append(features, np.zeros(delta_shape), axis=0)
        elif features.shape[0] > enforce_shape[0]:
            features = features[: enforce_shape[0], :]

    features[features == 0] = 1e-6
    return (np.log(features), float(new_input.shape[0]) / sample_rate)